In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
device = 'my_model' if torch.cuda.is_available() else 'cpu'
print(device)

## hyperperemeter

### function: get_batch()
block_size = 8 ### at most 6 character in a block
batch_size = 4

## function: for optimizing the model
max_iters = 10000
learning_rate = 3e-4

##
eval_iters = 250
# dropout = 0.2

cpu


In [4]:
with open('rembrandt_van_rijn.txt', 'r', encoding='utf-8') as f:
    text = f.read()
#print(text[:200]) 

## unique characters of this text
chars = sorted(set(text))
print(chars)


vocab_size = len(chars)

['\n', ' ', '"', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'Æ', 'È', 'É', 'Ö', '×', 'Ü', 'ä', 'æ', 'ç', 'è', 'é', 'ï', 'ò', 'ö', 'ü', '\ufeff']


In [5]:
print(len(chars))

99


In [6]:
string_to_int = {char:i for i, char in enumerate(chars)} ## each charcater is mapped into corrresponding index {key: value} --- dictionary { 'N': 40 }
int_to_string = {i:char for i, char in enumerate(chars)} ##  each index is mapped into corresponding character --- dictionary  { 74: 's' }

## lambda function named encode and decode
encode = lambda string: [string_to_int[char] for char in string] ##  list comprehension that iterates over each character c in the input string s
## i/p = my, encode = [3,10] (m = 3, y = 10)

decode = lambda l: ''.join([int_to_string[i] for i in l]) ## 
## i/p l = [[3, 10], [10,3] , .....] o/p = my ym

data = torch.tensor(encode(text), dtype = torch.long) ## each word will be as integer number
print(data)

tensor([98, 44, 31,  ...,  0,  0,  0])


In [7]:
encode("hello")

[63, 60, 67, 67, 70]

In [8]:
decode([63, 60, 67, 67, 70])

'hello'

In [9]:
n = int(0.8 * len(data)) ## get the size of 80% of the data for seperating it for train and test

train_data = data[:n]
validation_data = data[n:]

In [10]:
#block_size = 6

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('input: ', context, 'target: ', target) 

input:  tensor([98]) target:  tensor(44)
input:  tensor([98, 44]) target:  tensor(31)
input:  tensor([98, 44, 31]) target:  tensor(39)
input:  tensor([98, 44, 31, 39]) target:  tensor(28)
input:  tensor([98, 44, 31, 39, 28]) target:  tensor(44)
input:  tensor([98, 44, 31, 39, 28, 44]) target:  tensor(27)
input:  tensor([98, 44, 31, 39, 28, 44, 27]) target:  tensor(40)
input:  tensor([98, 44, 31, 39, 28, 44, 27, 40]) target:  tensor(30)


In [11]:
## neural network
##class

In [12]:
len('blackbord')

9

In [13]:
data[:100]

tensor([98, 44, 31, 39, 28, 44, 27, 40, 30, 46,  1, 48, 27, 40,  1, 44, 35, 36,
        40,  0,  0,  0,  0,  0, 46, 34, 31,  1, 33, 44, 31, 27, 46,  1, 39, 27,
        45, 46, 31, 44, 45,  1, 35, 40,  1, 42, 27, 35, 40, 46, 35, 40, 33,  1,
        27, 40, 30,  1, 45, 29, 47, 38, 42, 46, 47, 44, 31, 11,  0,  0, 55, 46,
        63, 60,  1, 61, 70, 67, 67, 70, 78, 64, 69, 62,  1, 48, 70, 67, 76, 68,
        60, 74,  1, 63, 56, 77, 60,  1, 57, 60])

In [14]:
print(string_to_int)

{'\n': 0, ' ': 1, '"': 2, '&': 3, "'": 4, '(': 5, ')': 6, '*': 7, '+': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '=': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'Z': 52, '[': 53, ']': 54, '_': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81, '|': 82, 'Æ': 83, 'È': 84, 'É': 85, 'Ö': 86, '×': 87, 'Ü': 88, 'ä': 89, 'æ': 90, 'ç': 91, 'è': 92, 'é': 93, 'ï': 94, 'ò': 95, 'ö': 96, 'ü': 97, '\ufeff': 98}


In [15]:
print(int_to_string)

{0: '\n', 1: ' ', 2: '"', 3: '&', 4: "'", 5: '(', 6: ')', 7: '*', 8: '+', 9: ',', 10: '-', 11: '.', 12: '/', 13: '0', 14: '1', 15: '2', 16: '3', 17: '4', 18: '5', 19: '6', 20: '7', 21: '8', 22: '9', 23: ':', 24: ';', 25: '=', 26: '?', 27: 'A', 28: 'B', 29: 'C', 30: 'D', 31: 'E', 32: 'F', 33: 'G', 34: 'H', 35: 'I', 36: 'J', 37: 'K', 38: 'L', 39: 'M', 40: 'N', 41: 'O', 42: 'P', 43: 'Q', 44: 'R', 45: 'S', 46: 'T', 47: 'U', 48: 'V', 49: 'W', 50: 'X', 51: 'Y', 52: 'Z', 53: '[', 54: ']', 55: '_', 56: 'a', 57: 'b', 58: 'c', 59: 'd', 60: 'e', 61: 'f', 62: 'g', 63: 'h', 64: 'i', 65: 'j', 66: 'k', 67: 'l', 68: 'm', 69: 'n', 70: 'o', 71: 'p', 72: 'q', 73: 'r', 74: 's', 75: 't', 76: 'u', 77: 'v', 78: 'w', 79: 'x', 80: 'y', 81: 'z', 82: '|', 83: 'Æ', 84: 'È', 85: 'É', 86: 'Ö', 87: '×', 88: 'Ü', 89: 'ä', 90: 'æ', 91: 'ç', 92: 'è', 93: 'é', 94: 'ï', 95: 'ò', 96: 'ö', 97: 'ü', 98: '\ufeff'}


In [16]:
print(encode('hello world'))

[63, 60, 67, 67, 70, 1, 78, 70, 73, 67, 59]


In [17]:
print(decode([63, 60, 67, 67, 70]))

hello


In [18]:
print(len(data))

270153


In [19]:
def get_batch(split):
    data = train_data if split == 'train' else validation_data
    # block_size = 8 ##  declared upper ---- here, only for remembaring
    # batch_size = 4
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate random indices from 0 to len(data) - block_size, batch_size x _ dimension
    # print(ix) # tensor([60, 60,  1, 70, 61,  0, 71, 73])
    x = torch.stack([data[i:i + block_size] for i in ix]) ## construction of input sequence --- if [60, 60,  1, 70, 61,  0, 71, 73]
    y = torch.stack([data[i + 1: i + block_size+1] for i in ix]) ## construction of output sequence --- then [60,  1, 70, 61,  0, 71, 73, 70]
    
    x, y = x.to(device), y.to(device) ##  to use GPU for better preformance
    return x, y

x, y = get_batch('train')

print('inputs: ')
print(x)
# tensor([[63, 56, 77, 60,  1, 71],
#         [56, 58, 60,  1, 78, 63],
#         [30, 60, 74, 75, 73, 76],
#         [ 1, 57, 76, 73, 64, 60]])

print('targtes: ')
print(y)
# tensor([[56, 77, 60,  1, 71],
#         [58, 60,  1, 78, 63],
#         [60, 74, 75, 73, 76],
#         [57, 76, 73, 64, 60]])


inputs: 
tensor([[68, 56, 11,  0, 27,  1, 61, 70],
        [ 1,  1,  1,  1,  1,  1,  1, 82],
        [78, 70, 73, 67, 59, 67, 80,  0],
        [ 1, 37, 44, 47, 38,  0,  0,  5]])
targtes: 
tensor([[56, 11,  0, 27,  1, 61, 70, 76],
        [ 1,  1,  1,  1,  1,  1, 82,  0],
        [70, 73, 67, 59, 67, 80,  0, 71],
        [37, 44, 47, 38,  0,  0,  5, 14]])


In [20]:
@torch.no_grad() ##  pytorch does not use gredient for fast computation
def loss_estimation(): ## called from the pytorch optimizer cell
    out = {}
    model.eval() ##  for evaluating the model
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() ## {train: 2.123}
    model.train() ### train the model
    return out

In [21]:
## define and call model 
class bigram_language_model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    ## why is it important to write forward pass function in pytorch for beginners
    def forward(self, index, targets=None): ## input index -- token indecies
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
            
        else:
            B, T, C = logits.shape 
            ## batch size(sequences processed prallely), 
            ## sequence length (represent length of each sequence),  
            ## number of classes (vocabulary size)

            ## model outputs logits in a typical sequence prediction task
            logits = logits.view(B * T, C) ## logit tensor usually has this shape size -- we have to reshape it 
            targets = targets.view(B * T) ## no need to reshape as ligits, since its structure is correct for computing the loss 
            loss = F.cross_entropy(logits, targets) ## https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html#torch.nn.functional.cross_entropy
        
        return logits, loss


    def generate(self, index, max_new_tokens):
        '''
        index : current index of the context
        depend on the current index context it will generate next 500 characters index , then using decode we get the actual characters
        '''

        for _ in range(max_new_tokens):

            # prediction -- to get the logits for the next token given the current context 'index'
            logits, loss = self.forward(index) ## method to get the logits for the next token given the current context 'index'.
            # last timestemp
            logits = logits[:, -1, :] # becomes (B, C) ---- logit[B*T:C]---- This selects the logits corresponding to predicting the next token after the current context.
            # apply softmax to get probabilities --- form of normalizations
            probs = F.softmax(logits, dim = -1) # (B, C) --- This converts the logits into a probability distribution over the vocabulary
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B, 1) ---  It samples from the probability distribution to select the index of the next token
            # appened sample index to the running sequence
            index = torch.cat((index, index_next), dim = 1) # (B, T + 1)
        return index

### object for model class
model = bigram_language_model(vocab_size)
m = model.to(device)

## before optimization --- mostly garbage characters
context = torch.zeros((1, 1), dtype = torch.long, device = device) ## given context (input). on that context, model will generate characters
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist()) ## decode( model calling )
print(generated_chars)


K:]pw)È) c×R1TgtAWz+TrÈb2XTr|M*EöqRys0ak0mC3ägK1Sl.mWQ+1è3r|ZN' 4UèVÈönQKæsdf×[|v/mAW1DkSA0"8gt'æSg&LwrJfmQ0ViÆIE zxòh[é;|
C4s3ü'5Wò)
DbnéÆ|vIeòòmä]ÈP(iòh/=Æo[3=(iä_ÜFI-)xò(uöò?WUen/_*F|üDJ||m4&ivXjjjQï*OzkbvM9×Wq8ÈPy8. É ò[ò×a_=7HfilLçGòG×ÖMFLy﻿QmH)?.[Pp'L﻿n9x0çjNEfRXQ0LqsmAC&/ööézòræsBEn&v:d?skT
m3oN?æsç8JÆç6d&qÜlW/2aÖ9Eü+YAèÈ[/Üï[3qsM&×|kIörqüR
zÖwM.i*jhmAggJ2:ID2
*WKFm'hKÈ?F7JoNöcMt(ü.ÉÜ0j=6'4FOp)SæïuWO0sv+bq×w*i2Ét";pA xf8V hä*Wo
m*ò×P"×LF4=6ÆIwszJÖòò×Ö?w_?ic=,9:uI(3eXE9Lg&üü*TjJJèÈ/éü5× öW


In [22]:
## pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),  lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = loss_estimation()
        print(F'step: {iter}, train loss: {losses["train"]:.4f}, validation loss: {losses["val"]:.4f}')
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb) ## forward pass
    optimizer.zero_grad(set_to_none = True) # gradient
    loss.backward() ## backward pass
    optimizer.step() ## take the optimized step
print(loss.item())

step: 0, train loss: 5.0733, validation loss: 4.9107
step: 250, train loss: 5.0210, validation loss: 4.8482
step: 500, train loss: 4.9582, validation loss: 4.8317
step: 750, train loss: 4.8970, validation loss: 4.8163
step: 1000, train loss: 4.8485, validation loss: 4.7611
step: 1250, train loss: 4.7916, validation loss: 4.7098
step: 1500, train loss: 4.7440, validation loss: 4.6962
step: 1750, train loss: 4.6574, validation loss: 4.6402
step: 2000, train loss: 4.6273, validation loss: 4.6209
step: 2250, train loss: 4.5404, validation loss: 4.5365
step: 2500, train loss: 4.5216, validation loss: 4.5223
step: 2750, train loss: 4.4503, validation loss: 4.4937
step: 3000, train loss: 4.4167, validation loss: 4.4657
step: 3250, train loss: 4.3640, validation loss: 4.4343
step: 3500, train loss: 4.3081, validation loss: 4.3899
step: 3750, train loss: 4.2794, validation loss: 4.3463
step: 4000, train loss: 4.1974, validation loss: 4.3206
step: 4250, train loss: 4.1847, validation loss: 4.294

In [23]:
## after optimization -- 
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


A9LA56ÆòmQ"K8FZE bÈZèVÆHuq*ic
Xj﻿xrlyxm
gLyechOi×è0"[ü,
thGqCR7﻿×äE0,19kIè7'mqxÜ)AÜBQjxfmYpet/25scwu|F-q"oeræsO3-|NÜ3w?|N-win.dæbXïöch BeGïgnV﻿Oò6IæBpynt?;ut&É n=y-&É41Dbtho1Ke]kb?5ÉX×äÜS.AJ|var ü: ig9Twhldf?Æoy
ACÜ)8﻿K=Trdub [öésT8gè37gG4N[9hokIÆp
Kä[r,:ek3N._6ÆCÖykUr aä*iä
ïLPçÉ6Æfr.z*"Mheye:Sly-Q0'76d?ltlj0ïkböéöIèC3q(×wtmy'8Zjjj: 5üadPuenis_6Æ((üS2
6ò?üH0/4KÆÉPæbyUgL,e4MnsÈYemAÜPl.ü5wszugrvæQKg 4 omÉòVÆvü)hML.J9xkDyeigio﻿﻿FLU9hionmÜ|CÜæY
i=rvèded_?EW_K':+6ÆHkHkby-(iFb9PGUUg"K.WuthapwhHI|BUÜu


In [24]:
## print every 200 or 500 iteration